#### Copyright (C) 2022 Sobhan Moradian Daghigh
#### Date: 2/2/2022

### Import Libraries

In [66]:
import numpy as np
import pandas as pd
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import string
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk import pos_tag
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')

from IPython.display import display

import import_ipynb, importlib, sys
import feature_selection_methods as FS
import classifier_models as models
def reload_moduls():
    importlib.reload(sys.modules['feature_selection_methods'])
    importlib.reload(sys.modules['classifier_models'])
    import feature_selection_methods as FS
    import classifier_models as models
    
    return FS, models

### Loading Datasets

In [2]:
def load_datasets(path):
    datasets = []
    print('Loading the datasets..\n')
    for root, dirs, files in os.walk(path):
        for i, file in enumerate(files):
            print(' |_  {}. {} loaded'.format(i + 1, file.split('.')[0]))
            datasets.append(pd.read_csv(os.path.join(root, file), names=['sentence', 'label'], sep='\t'))
    return datasets

In [3]:
datasets_name = ['Amazon', 'IMDB  ', 'Yelp  ']
datasets = load_datasets('./Datasets/')

Loading the datasets..

 |_  1. amazon loaded
 |_  2. imdb loaded
 |_  3. yelp loaded


In [4]:
# Amazon
datasets[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  1000 non-null   object
 1   label     1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [5]:
# IMDB
datasets[1].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  1000 non-null   object
 1   label     1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [6]:
# Yelp
datasets[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  1000 non-null   object
 1   label     1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


### Make a preprocess on the comments to improve quality and remove unneccessary words
##### 1. convert all words to lowercase
##### 2. remove punctuations
##### 3. tonekize words
##### 4. remove stopwords
##### 5. use porterStemmer to have stemming words

In [7]:
def nlprocess(dataset):
    
    preprocessed_comments = []
    for text in dataset:
        
        #Lowercase
        text = text.lower()

        # Removing Punctuation
        text = "".join([char for char in text if char not in string.punctuation])

        # Word Tokenization
        text = word_tokenize(text)

        # Stopword Filtering
        stop_words = stopwords.words('english')
        text = [word for word in text if word not in stop_words]

        # Stemming
        porter = PorterStemmer()
        text = [porter.stem(word) for word in text]

        text = ' '.join(text)

        preprocessed_comments.append(text)
        
    return preprocessed_comments

### Splitting the datasets into Train and Test

In [8]:
def spliter(datasets):
    x_train_datasets, x_test_datasets, y_train_datasets, y_test_datasets = [], [], [], []
    for dataset in datasets:
        x_train, x_test, y_train, y_test = train_test_split(dataset.sentence, dataset.label, test_size=0.3, random_state=0)
        x_train_datasets.append(nlprocess(x_train))
        x_test_datasets.append(nlprocess(x_test))
        y_train_datasets.append(y_train)
        y_test_datasets.append(y_test)
    return x_train_datasets, x_test_datasets, y_train_datasets, y_test_datasets

In [9]:
x_train_datasets, x_test_datasets, y_train_datasets, y_test_datasets = spliter(datasets)

### Feature Extraction using TF-IDF

In [10]:
def feature_extractor(dataset):
    vectorizer = TfidfVectorizer()
    vectorizer.fit(dataset)
    return vectorizer

In [11]:
def get_top_frequent_features(vectorizer):
    df = pd.DataFrame({'words': vectorizer.vocabulary_.keys(), 'counts': vectorizer.vocabulary_.values()})
    print('\n------ {} ------'.format(datasets_name[i]))
    display(df.sort_values(by=['counts'], ascending=False).head())

#### Top 5 frequent words of each dataset to get a full feeling of the problem

In [12]:
# Display
vectorizers = []
for i, dataset in enumerate(x_train_datasets):
    vectorizer = feature_extractor(dataset)
    get_top_frequent_features(vectorizer)
    vectorizers.append(vectorizer)


------ Amazon ------


,words,counts
526,zero,1195
561,yet,1194
929,yearsgreat,1193
423,year,1192
1180,ye,1191



------ IMDB   ------


,words,counts
545,zombiez,2056
2052,zombiestud,2055
1140,zombi,2054
1659,yun,2053
1644,youv,2052



------ Yelp   ------


,words,counts
203,zero,1327
508,yummi,1326
1155,yukon,1325
792,yucki,1324
584,your,1323


### Logistic Regression
#### - Without FS

In [13]:
print("Accuracy:")
clfs = []
vectorized_trains = []
vectorized_tests = []

for i, (vec, x_tr, x_ts, y_tr, y_ts) in enumerate(zip(vectorizers, x_train_datasets, x_test_datasets, y_train_datasets, y_test_datasets)):
    X_train = vec.transform(x_tr)
    X_test = vec.transform(x_ts)

    vectorized_trains.append(X_train)
    vectorized_tests.append(X_test)
    
    clf = LogisticRegression()
    clf.fit(X_train, y_tr)
    clfs.append(clf)
    score = clf.score(X_test, y_ts)
    
    print(' |_  {}: {:.3f}'.format(datasets_name[i], score))

Accuracy:
 |_  Amazon: 0.790
 |_  IMDB  : 0.787
 |_  Yelp  : 0.750


### Apply the selected features to datasets

In [14]:
def apply_selected_features(datasets, tops):
    filter_datasets = []
    for dataset, top in zip(datasets, tops):
        filter_dataset = np.array(np.take(dataset.todense(), top, axis=1))
        filter_datasets.append(filter_dataset)
    return filter_datasets

### Display the top features

In [15]:
def display_tops(tops, vectorizers):
    for i, (top, vec) in enumerate(zip(tops, vectorizers)):
        name = np.array(vec.get_feature_names_out())[top].tolist()
        print('\n{}:'.format(datasets_name[i]))
        display(pd.DataFrame(name).head())

In [99]:
# Including the features which are selected by different FS methods
fs_tops = []
fs_methods_name = ['Odd-Ratio', 'Chi-Square', 'Count-Difference']

### 1. Odd Ratio FS

In [100]:
# Reload the sub_modules to apply changes
FS, models = reload_moduls()

odd_ratio_tops = FS.odd_ratio(clfs)
fs_tops.append(odd_ratio_tops)
display_tops(odd_ratio_tops, vectorizers)

importing Jupyter notebook from feature_selection_methods.ipynb
importing Jupyter notebook from classifier_models.ipynb
(Main, Selected) Features size:
 |_ Amazon: (1196, 24)
 |_ IMDB  : (2057, 22)
 |_ Yelp  : (1328, 18)

Amazon:


,0
0,great
1,good
2,excel
3,nice
4,love



IMDB  :


,0
0,love
1,great
2,good
3,excel
4,wonder



Yelp  :


,0
0,great
1,love
2,good
3,delici
4,amaz


### 2. Chi-Square FS

In [101]:
# Reload the sub_modules to apply changes
FS, models = reload_moduls()

chi_tops = FS.chi_square(vectorized_trains, y_train_datasets)
fs_tops.append(chi_tops)
display_tops(chi_tops, vectorizers)

importing Jupyter notebook from feature_selection_methods.ipynb
importing Jupyter notebook from classifier_models.ipynb
(Main, Selected) Features size:
 |_ Amazon: (1196, 148)
 |_ IMDB  : (2057, 154)
 |_ Yelp  : (1328, 151)

Amazon:


,0
0,great
1,good
2,price
3,disappoint
4,excel



IMDB  :


,0
0,bad
1,love
2,great
3,aw
4,excel



Yelp  :


,0
0,great
1,delici
2,love
3,good
4,amaz


### 3. Count Difference FS

In [102]:
# Reload the sub_modules to apply changes
FS, models = reload_moduls()

cd_tops = FS.count_diff(vectorized_trains, y_train_datasets)
fs_tops.append(cd_tops)
display_tops(cd_tops, vectorizers)

importing Jupyter notebook from feature_selection_methods.ipynb
importing Jupyter notebook from classifier_models.ipynb
(Main, Selected) Features size:
 |_ Amazon: (1196, 190)
 |_ IMDB  : (2057, 204)
 |_ Yelp  : (1328, 202)

Amazon:


,0
0,great
1,good
2,price
3,excel
4,disappoint



IMDB  :


,0
0,bad
1,great
2,love
3,aw
4,even



Yelp  :


,0
0,great
1,good
2,love
3,delici
4,amaz


### ------------------------------------------------------------    Models    ---------------------------------------------------

### Logistic Regression

In [104]:
# Reload the sub_modules to apply changes
FS, models = reload_moduls()

for i, tops in enumerate(fs_tops):
    print('\nFS: {}'.format(fs_methods_name[i]))
    filtered_datasets_trains = apply_selected_features(vectorized_trains, tops)
    filtered_datasets_tests = apply_selected_features(vectorized_tests, tops)
    models.logestic_regression(filtered_datasets_trains, filtered_datasets_tests, y_train_datasets, y_test_datasets)

importing Jupyter notebook from feature_selection_methods.ipynb
importing Jupyter notebook from classifier_models.ipynb

FS: Odd-Ratio
Accuracy:
 |_  Amazon: 0.737
 |_  IMDB  : 0.647
 |_  Yelp  : 0.693

FS: Chi-Square
Accuracy:
 |_  Amazon: 0.780
 |_  IMDB  : 0.713
 |_  Yelp  : 0.753

FS: Count-Difference
Accuracy:
 |_  Amazon: 0.773
 |_  IMDB  : 0.753
 |_  Yelp  : 0.727


### Linear SVM

In [106]:
# Reload the sub_modules to apply changes
FS, models = reload_moduls()

for i, tops in enumerate(fs_tops):
    print('\nFS: {}'.format(fs_methods_name[i]))
    filtered_datasets_trains = apply_selected_features(vectorized_trains, tops)
    filtered_datasets_tests = apply_selected_features(vectorized_tests, tops)
    models.linear_svm(filtered_datasets_trains, filtered_datasets_tests, y_train_datasets, y_test_datasets)

importing Jupyter notebook from feature_selection_methods.ipynb
importing Jupyter notebook from classifier_models.ipynb

FS: Odd-Ratio
Accuracy:
 |_  Amazon: 0.737
 |_  IMDB  : 0.650
 |_  Yelp  : 0.697

FS: Chi-Square
Accuracy:
 |_  Amazon: 0.787
 |_  IMDB  : 0.703
 |_  Yelp  : 0.750

FS: Count-Difference
Accuracy:
 |_  Amazon: 0.743
 |_  IMDB  : 0.753
 |_  Yelp  : 0.713


### RBF SVM

In [123]:
# Reload the sub_modules to apply changes
FS, models = reload_moduls()

for i, tops in enumerate(fs_tops):
    print('\nFS: {}'.format(fs_methods_name[i]))
    filtered_datasets_trains = apply_selected_features(vectorized_trains, tops)
    filtered_datasets_tests = apply_selected_features(vectorized_tests, tops)
    models.rbf_svm(filtered_datasets_trains, filtered_datasets_tests, y_train_datasets, y_test_datasets)

importing Jupyter notebook from feature_selection_methods.ipynb
importing Jupyter notebook from classifier_models.ipynb

FS: Odd-Ratio
Accuracy:
 |_  Amazon: 0.740
 |_  IMDB  : 0.650
 |_  Yelp  : 0.693

FS: Chi-Square
Accuracy:
 |_  Amazon: 0.770
 |_  IMDB  : 0.703
 |_  Yelp  : 0.757

FS: Count-Difference
Accuracy:
 |_  Amazon: 0.767
 |_  IMDB  : 0.763
 |_  Yelp  : 0.733


### Decision Tree

In [133]:
# Reload the sub_modules to apply changes
FS, models = reload_moduls()

for i, tops in enumerate(fs_tops):
    print('\nFS: {}'.format(fs_methods_name[i]))
    filtered_datasets_trains = apply_selected_features(vectorized_trains, tops)
    filtered_datasets_tests = apply_selected_features(vectorized_tests, tops)
    models.decision_tree(filtered_datasets_trains, filtered_datasets_tests, y_train_datasets, y_test_datasets)

importing Jupyter notebook from feature_selection_methods.ipynb
importing Jupyter notebook from classifier_models.ipynb

FS: Odd-Ratio
Accuracy:
 |_  Amazon: 0.723
 |_  IMDB  : 0.643
 |_  Yelp  : 0.663

FS: Chi-Square
Accuracy:
 |_  Amazon: 0.733
 |_  IMDB  : 0.617
 |_  Yelp  : 0.683

FS: Count-Difference
Accuracy:
 |_  Amazon: 0.743
 |_  IMDB  : 0.627
 |_  Yelp  : 0.673
